In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

import shared_utils
import branca
from utils import *

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### Get and transform test data (Long Beach Transit, 170)

In [2]:
lbt_itp_id = 170

In [110]:
%%sql -o lbt_positions

# get all vehicle positions on selected dates, for the feed with itp id 170, and url number 0
SELECT *
FROM `cal-itp-data-infra.gtfs_rt.vehicle_positions`
WHERE _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-01_170_0.parquet"
    OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-11-30_170_0.parquet"
    # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-03_170_0.parquet"
    # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-04_170_0.parquet"
    # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-05_170_0.parquet"
    # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-06_170_0.parquet"
    # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-07_170_0.parquet"
ORDER BY header_timestamp

,calitp_itp_id,calitp_url_number,entity_id,vehicle_id,vehicle_trip_id,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp
0,170,0,2403,None,9288416,1638230352,33.775260,-118.152000,1638230404
1,170,0,2404,None,9288666,1638230357,33.768030,-118.191220,1638230404
2,170,0,2407,None,9288521,1638230373,33.775394,-118.141396,1638230404
3,170,0,2408,None,9288668,1638230383,33.789986,-118.188990,1638230404
4,170,0,2410,None,9288518,1638230371,33.775420,-118.182076,1638230404
...,...,...,...,...,...,...,...,...,...
573955,170,0,1834,None,9289002,1638386137,33.832580,-118.168070,1638386161
573956,170,0,1836,None,9289440,1638386157,33.874330,-118.190254,1638386161
573957,170,0,1837,None,9289382,1638386158,33.768020,-118.190710,1638386161
573958,170,0,1839,None,9288160,1638386149,33.769096,-118.189520,1638386161


In [111]:
# lbt_positions.to_parquet(f'{GCS_FILE_PATH}lbt_positions.parquet')

In [112]:
# lbt_positions = pd.read_parquet(f'{GCS_FILE_PATH}lbt_positions.parquet')

#### Reformat timestamps

In [113]:
## set system time
os.environ['TZ'] = 'America/Los_Angeles'
time.tzset()
time.tzname

('PST', 'PDT')

In [114]:
lbt_positions.vehicle_timestamp = lbt_positions.vehicle_timestamp.apply(convert_ts)
lbt_positions.header_timestamp = lbt_positions.header_timestamp.apply(convert_ts)

In [115]:
pch_routes = ('171', '172', '173', '174', '175')

In [116]:
# min_date, max_date = ('2021-11-01', '2021-12-09')

# lbt_trips = (tbl.views.gtfs_schedule_fact_daily_trips()
#     # >> filter(_.calitp_extracted_at <= min_date, _.calitp_deleted_at > max_date)
#     >> filter(_.calitp_itp_id == lbt_itp_id)
#     >> filter(_.service_date == max_date)
#     >> filter(_.is_in_service == True)
#     >> select(_.trip_key, _.service_date)
#     >> inner_join(_, tbl.views.gtfs_schedule_dim_trips(), on = 'trip_key')
#     >> select(_.calitp_itp_id, _.calitp_url_number,
#               _.date == _.service_date,
#               _.trip_key, _.trip_id, _.route_id, _.direction_id,
#               _.shape_id, _.calitp_extracted_at, _.calitp_deleted_at)
#     # >> filter(_.route_id.isin(pch_routes))

#     >> collect()
#     )

In [117]:
# lbt_trips.to_parquet(f'{GCS_FILE_PATH}lbt_trips.parquet')

In [118]:
lbt_trips = pd.read_parquet(f'{GCS_FILE_PATH}lbt_trips.parquet')

In [119]:
# stop_times = (tbl.gtfs_schedule.stop_times()
#      >> filter(_.calitp_itp_id == 170)
#      # >> filter(_.trip_id == '9288623')
#      >> mutate(stop_sequence = _.stop_sequence.astype(int)) ## in SQL!
#      >> arrange(_.stop_sequence)
#      >> collect()
#     )

In [120]:
# stop_times.to_parquet(f'{GCS_FILE_PATH}lbt_st.parquet')

In [121]:
stop_times = pd.read_parquet(f'{GCS_FILE_PATH}lbt_st.parquet')

In [122]:
# stops = (tbl.gtfs_schedule.stops()
#      >> filter(_.calitp_itp_id == 170)
#      # >> filter(_.trip_id == '9288623')
#      # >> mutate(stop_sequence = _.stop_sequence.astype(int)) ## in SQL!
#      # >> arrange(_.stop_sequence)
#      >> select(_.calitp_itp_id, _.calitp_url_number, _.stop_id,
#               _.stop_lat, _.stop_lon, _.stop_name)
#      >> collect()
#     )

# stops = gpd.GeoDataFrame(stops, geometry=gpd.points_from_xy(stops.stop_lon, stops.stop_lat),
#                         crs='EPSG:4326').to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [123]:
# shared_utils.utils.geoparquet_gcs_export(stops, GCS_FILE_PATH, 'lbt_stops')

In [124]:
stops = gpd.read_parquet(f'{GCS_FILE_PATH}lbt_stops.parquet')

In [125]:
trips_positions_joined = (lbt_trips
                        # >> filter(_.route_id.isin(pch_routes))
                        >> inner_join(_, (lbt_positions >> rename(trip_id = 'vehicle_trip_id')),
                                      on= ['trip_id', 'calitp_itp_id', 'calitp_url_number'])
                       )

In [126]:
trips_positions_joined.vehicle_timestamp.max()

Timestamp('2021-12-01 11:15:59')

In [127]:
trips_positions_joined.vehicle_timestamp.min()

Timestamp('2021-11-29 15:57:13')

In [128]:
trips_positions_joined = gpd.GeoDataFrame(trips_positions_joined,
                                geometry=gpd.points_from_xy(trips_positions_joined.vehicle_position_longitude,
                                                            trips_positions_joined.vehicle_position_latitude),
                                crs=shared_utils.geography_utils.WGS84).to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [22]:
# lbt_routelines = shared_utils.geography_utils.make_routes_shapefile(['170'], ## LBT
#                                 shared_utils.geography_utils.CA_NAD83Albers)

In [23]:
# shared_utils.utils.geoparquet_gcs_export(lbt_routelines, GCS_FILE_PATH, 'lbt_routelines')

In [24]:
lbt_routelines = gpd.read_parquet(f'{GCS_FILE_PATH}lbt_routelines.parquet')

## Vehicle Positions Trip class

#### Examples

In [25]:
example_trip = trips_positions_joined >> filter(_.trip_id == '9288623')

In [26]:
example_vp_trip = rt.VehiclePositionsTrip(example_trip, lbt_routelines)

In [27]:
example_vp_trip.vehicle_positions.head(3)

,vehicle_timestamp,header_timestamp,geometry,shape_meters,last_time,last_loc,secs_from_last,meters_from_last,progressed,speed_from_last
0,2021-11-30 17:02:12,2021-11-30 17:02:37,POINT (174550.467 -468602.675),74.936069,NaT,NaN,NaN,NaN,False,NaN
1,2021-11-30 17:03:13,2021-11-30 17:03:37,POINT (174199.373 -468606.652),426.501155,2021-11-30 17:02:12,74.936069,61.0,351.565087,True,5.763362
2,2021-11-30 17:04:13,2021-11-30 17:04:37,POINT (173950.148 -468607.933),676.459161,2021-11-30 17:03:13,426.501155,60.0,249.958006,True,4.165967


In [28]:
example_vp_trip.time_at_position(550) ## interpolate time at 550 meters from start of route (shape_meters in df^)

Timestamp('2021-11-30 17:03:42.644702')

In [29]:
# example_vp_trip.detailed_speed_map()

## General RT Analysis class

In [30]:
example_trips = (trips_positions_joined
 >> filter(_.route_id == '175', _.direction_id == '0') ## towards Villages at Cabrillo
 >> group_by(_.trip_id)
 >> summarize(max_time = _.vehicle_timestamp.max())
 >> arrange(_.max_time)
 >> head(6) ## afternoon/evening
)
example_trips

,trip_id,max_time
8,9288619,2021-11-30 16:37:23
9,9288621,2021-11-30 17:17:58
10,9288623,2021-11-30 17:56:04
11,9288625,2021-11-30 18:35:49
12,9288627,2021-11-30 19:24:12
13,9288629,2021-11-30 19:59:08


In [31]:
%%capture
## suppress warnings from location data inaccuracies
## generate test analysis with a small subset of LBT trips on a particular route, time, direction
test_analysis = rt.RtAnalysis(trips_positions_joined, stop_times, stops, lbt_routelines, example_trips.trip_id)

In [32]:
%%capture
## suppress warnings from start/end of routes
all_delays = test_analysis.generate_delay_view()

In [33]:
all_delays.head(3)

,arrival_time,actual_time,delay,stop_id,trip_id,shape_id,direction_id,direction,stop_sequence,route_id,shape_meters,date,geometry
148,2021-11-30 16:58:10,2021-11-30 17:03:30.892511,0 days 00:05:20.892511,1027,9288623,1750012,0,Westbound,2,175,501.040761,2021-12-01,POINT (174124.462 -468604.478)
145,2021-11-30 17:39:10,2021-11-30 17:43:07.179287,0 days 00:03:57.179287,1027,9288625,1750012,0,Westbound,2,175,501.040761,2021-12-01,POINT (174124.462 -468604.478)
86,2021-11-30 17:00:17,2021-11-30 17:05:15.130531,0 days 00:04:58.130531,0540,9288623,1750012,0,Westbound,3,175,991.109291,2021-12-01,POINT (173921.407 -468897.107)


In [34]:
endpoint_delay_sum = (all_delays
 >> group_by(_.trip_id)
 >> filter(_.stop_sequence == _.stop_sequence.max())
 >> ungroup()
 >> mutate(total_delay = _.delay.sum())
)

In [35]:
endpoint_delay_sum

,arrival_time,actual_time,delay,stop_id,trip_id,shape_id,direction_id,direction,stop_sequence,route_id,shape_meters,date,geometry,total_delay
0,2021-11-30 16:58:00,2021-11-30 17:10:28.976334,0 days 00:12:28.976334,3150,9288621,1750012,0,Westbound,29,175,11480.641185,2021-12-01,POINT (164664.672 -467611.924),0 days 01:04:39.968770
1,2021-11-30 18:56:00,2021-11-30 19:20:48.778780,0 days 00:24:48.778780,3150,9288627,1750012,0,Westbound,29,175,11480.641185,2021-12-01,POINT (164664.672 -467611.924),0 days 01:04:39.968770
2,2021-11-30 17:34:39,2021-11-30 17:44:59.937535,0 days 00:10:20.937535,3049,9288623,1750012,0,Westbound,28,175,10745.097772,2021-12-01,POINT (164733.079 -467919.985),0 days 01:04:39.968770
3,2021-11-30 16:14:39,2021-11-30 16:28:44.013076,0 days 00:14:05.013076,3049,9288619,1750012,0,Westbound,28,175,10745.097772,2021-12-01,POINT (164733.079 -467919.985),0 days 01:04:39.968770
4,2021-11-30 18:14:39,2021-11-30 18:16:47.014459,0 days 00:02:08.014459,3049,9288625,1750012,0,Westbound,28,175,10745.097772,2021-12-01,POINT (164733.079 -467919.985),0 days 01:04:39.968770
5,2021-11-30 19:37:39,2021-11-30 19:38:27.248586,0 days 00:00:48.248586,3049,9288629,1750012,0,Westbound,28,175,10745.097772,2021-12-01,POINT (164733.079 -467919.985),0 days 01:04:39.968770


In [36]:
all_summary = test_analysis.generate_delay_summary()

In [37]:
all_summary.head(3)

,stop_id,geometry,stop_sequence,avg_delay,max_delay
0,1027,POINT (174124.462 -468604.478),2,0 days 00:04:39.035899,0 days 00:05:20.892511
1,0540,POINT (173921.407 -468897.107),3,0 days 00:04:00.527345,0 days 00:04:58.130531
2,0542,POINT (173328.363 -468647.513),4,0 days 00:01:40.985010250,0 days 00:03:24.379293


In [38]:
# test_analysis.map_stop_delays(how = 'average') ## to refine, add hover, legend, etc...

In [39]:
test_analysis.map_segment_speeds(how = 'low_speeds')

/opt/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


1750012 (27, 14)


In [40]:
# test_analysis.map_stop_delays()

## Sandbox

In [41]:
trips_positions_joined >> filter(_.direction_id == '0') >> count(_.trip_id)

,trip_id,n
0,9288152,204
1,9288153,354
2,9288154,259
3,9288155,247
4,9288156,272
...,...,...
633,9296926,61
634,9296927,103
635,9296928,103
636,9297176,68


In [132]:
trips_positions_joined.vehicle_timestamp.iloc[0].day

30

In [133]:
nov_30 = (trips_positions_joined
 # >> filter(_.direction_id == '0') ## towards 
 # >> filter((_.vehicle_timestamp.apply(lambda x: x.hour < 20))
 #           & (_.vehicle_timestamp.apply(lambda x: x.hour > 16))) ## 4-8pm
 >> mutate(timestamp_day = _.vehicle_timestamp.apply(lambda x: x.day))
 # >> mutate(timestamp_hour = _.vehicle_timestamp.apply(lambda x: x.hour))
 # >> filter((_.timestamp_hour < 20)
 #           & (_.timestamp_hour > 16)) ## 4-8pm >> group_by(_.trip_id)
 >> filter(_.timestamp_day == 30)
 >> group_by(_.trip_id)
 >> summarize(max_time = _.vehicle_timestamp.max())
 >> arrange(_.max_time)
 # >> head(40) ## afternoon/evening
)
nov_30

,trip_id,max_time
1359,9296591,2021-11-30 05:13:58
1360,9296592,2021-11-30 05:24:45
315,9288467,2021-11-30 05:26:39
900,9289180,2021-11-30 05:39:54
995,9289275,2021-11-30 05:42:23
...,...,...
236,9288388,2021-11-30 23:02:54
1199,9289719,2021-11-30 23:04:21
969,9289249,2021-11-30 23:08:42
911,9289191,2021-11-30 23:09:27


In [134]:
test_many = rt.RtAnalysis(trips_positions_joined,
                           stop_times,
                           stops,
                           lbt_routelines,
                           nov_30.trip_id)

check location data for trip 9296591
check location data for trip 9296592


/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


check location data for trip 9288467
check location data for trip 9289180


/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


check location data for trip 9289275


...

/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


check location data for trip 9289191


/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


check location data for trip 9288386


/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [135]:
%%capture --no-stderr
test_many.generate_delay_view()

In [136]:
test_many.trip_ids.nunique()

1676

In [137]:
test_many.delay_view.trip_id.nunique()

1231

In [138]:
test_many.delay_view.shape_id.nunique()

92

In [139]:
test_many.delay_view.head(3)

,arrival_time,actual_time,delay,stop_id,trip_id,shape_id,direction_id,direction,stop_sequence,route_id,shape_meters,date,geometry
42978,2021-11-30 05:59:59,2021-11-30 06:00:19.267516,0 days 00:00:20.267516,1573,9288199,1910295,0,Westbound,2,191,559.250839,2021-12-01,POINT (179249.800 -461991.129)
14349,2021-11-30 11:17:12,2021-11-30 11:19:28.364062,0 days 00:02:16.364062,0065,9288250,1910292,1,Eastbound,2,191,541.755286,2021-12-01,POINT (167263.401 -470018.350)
14285,2021-11-30 11:47:12,2021-11-30 11:52:02.115455,0 days 00:04:50.115455,0065,9288252,1910292,1,Eastbound,2,191,541.755286,2021-12-01,POINT (167263.401 -470018.350)


In [140]:
endpoint_delay_sum = (test_many.delay_view
 >> group_by(_.trip_id)
 >> filter(_.stop_sequence == _.stop_sequence.max())
 >> ungroup()
 >> mutate(total_delay = _.delay.sum())
)

In [141]:
endpoint_delay_sum.head(3)

,arrival_time,actual_time,delay,stop_id,trip_id,shape_id,direction_id,direction,stop_sequence,route_id,shape_meters,date,geometry,total_delay
0,2021-11-30 15:14:34,2021-11-30 15:18:15.632030,0 days 00:03:41.632030,4018,9296624,460067,0,Westbound,29,46,6580.90515,2021-12-01,POINT (167729.882 -470046.387),1 days 22:45:27.085280
1,2021-11-30 13:34:34,2021-11-30 13:32:13.179104,-1 days +23:57:39.179104,4018,9296619,460067,0,Westbound,29,46,6580.90515,2021-12-01,POINT (167729.882 -470046.387),1 days 22:45:27.085280
2,2021-11-30 05:43:58,2021-11-30 05:44:01.876089,0 days 00:00:03.876089,4018,9296594,460067,0,Westbound,29,46,6580.90515,2021-12-01,POINT (167729.882 -470046.387),1 days 22:45:27.085280


In [142]:
starts = (test_many.delay_view
 >> group_by(_.trip_id)
 >> filter(_.stop_sequence == _.stop_sequence.min())
 >> ungroup()
 # >> mutate(total_delay = _.delay.sum())
 >> select(_.start_time == _.actual_time, _.trip_id, _.start_meters == _.shape_meters)
)

In [143]:
speed_and_delay = endpoint_delay_sum >> inner_join(_, starts, on='trip_id')

In [144]:
speed_and_delay['seconds'] = speed_and_delay.apply(
    lambda x: (x.actual_time - x.start_time).seconds, axis=1)

In [145]:
speed_and_delay['trip_speed'] = (speed_and_delay.shape_meters - speed_and_delay.start_meters) / speed_and_delay.seconds

In [146]:
speed_and_delay['trip_speed_mph'] = speed_and_delay.trip_speed * MPH_PER_MPS

In [147]:
speed_and_delay['trip_speed_mph'].mean()

12.568541498779783

In [ ]:
m = test_many.map_segment_speeds(how = 'average')

In [ ]:
colorscale = branca.colormap.step.RdYlGn_10.scale(vmin=0, 
 vmax=30)
colorscale.caption = "Speed (miles per hour)"

In [ ]:
m = test_many._generate_segment_map(colorscale)

In [ ]:
type(m)

In [ ]:
# m

In [ ]:
# m

In [ ]:
m.save('lbt_speed_map.html')

In [ ]:
m = example_vp_trip.detailed_speed_map()

In [ ]:
m

In [ ]:
m.save('lbt_detailled_speeds.html')